In [ ]:
--Create a schema
CREATE SCHEMA Users;
GO


In [ ]:
--Grant Permissions 
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::Users TO public;
GO


In [ ]:
--Move table to another schema
ALTER SCHEMA Users TRANSFER dbo.UserMaster;


In [ ]:
--To delete a schema you have to make sure that it is empty
DECLARE @OldSchemaName NVARCHAR(128) = 'OldSchema'; -- Replace with your current schema name
DECLARE @NewSchemaName NVARCHAR(128) = 'NewSchema'; -- Replace with your new schema name
DECLARE @sql NVARCHAR(MAX) = '';

-- Transfer Tables
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.tables
WHERE schema_id = SCHEMA_ID(@OldSchemaName);

-- Transfer Views
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.views
WHERE schema_id = SCHEMA_ID(@OldSchemaName);

-- Transfer Stored Procedures
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.procedures
WHERE schema_id = SCHEMA_ID(@OldSchemaName);

-- Transfer Functions
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.objects
WHERE schema_id = SCHEMA_ID(@OldSchemaName)
    AND type IN ('FN', 'TF', 'IF', 'FS', 'FT'); -- Scalar, Table-valued, and Aggregate Functions

-- Transfer Synonyms
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.synonyms
WHERE schema_id = SCHEMA_ID(@OldSchemaName);

-- Transfer Sequences
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.sequences
WHERE schema_id = SCHEMA_ID(@OldSchemaName);

-- Transfer Types
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + name + '];' + CHAR(13)
FROM sys.types
WHERE schema_id = SCHEMA_ID(@OldSchemaName) AND is_user_defined = 1;

-- Transfer XML Schema Collections
SELECT @sql += 'ALTER SCHEMA [' + @NewSchemaName + '] TRANSFER [' + @OldSchemaName + '].[' + XML_collection_name + '];' + CHAR(13)
FROM sys.xml_schema_collections
WHERE schema_id = SCHEMA_ID(@OldSchemaName) AND is_xml_namespace_default = 0;

EXEC sp_executesql @sql;--move all tables to new schema

--then drop the schema
DROP SCHEMA OldSchema;


In [ ]:
--To revoke the permission 
REVOKE [permission] ON [securable] FROM [principal];
--permission: The type of permission you want to revoke (e.g., SELECT, INSERT, UPDATE, etc.).
--securable: The object or scope for which the permission was granted (e.g., table, view, schema).
--principal: The user, role, or group from whom you want to revoke the permission.
